In [ ]:
#pip install -U folium

# Week 0 - actual 2nd course assignment starts here

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### The postcodes for Toronto are avaible at [[https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)]

There are several ways to get the data from the table such as beuatiful soup or reading the URL inot a dataframe.  The Capstone  instructions said d I can use any method so I simply selected and pasted the table area into a LibreOffice documnent then from that cut and pasted the data into a spreadsheet and saved as a CSV file.  This I then saved in my Kaggle data reared and opened into a Pandas frame.

How ever we now have to clean the data by dropingpost codes not assigned and combining post code data listed twice usinf a comma.

In [ ]:
df = pd.read_csv('/kaggle/input/post-codes-toronto/pstalcodes.csv') #This is the raw data before cleaning
df.head()

We have to remove any not assigned post codes

In [ ]:
df = df[df.Borough != 'Not assigned']
df.head()

That looks good so far but need to find duplicate Post Code/s

In [ ]:
#looking for duplicate Post Codes
df[df['Post Code'].duplicated() == True]


#### As you can see no duplicted post codes

****Looking for none assiged neighbour hoods[](http://)

In [ ]:
df[df.Neighbourhood == 'Not assigned']


**** Again none so just maybe we have clean data

In [ ]:
print(df.shape)

## 103 rows 3 columns

In [ ]:
#In the case we want to display all the rows
#pd.set_option('display.max_rows', df.shape[0]+1)


In [ ]:
df.head()

# Now we add coordinates from CSV# 

In [ ]:
df2 = pd.read_csv('/kaggle/input/coordinates-toronto/Coord.csv') #This is the raw data before cleaning
df2.head()

Add two new colomns to take the coordiantes

In [ ]:
df2 = df2.rename(columns={"Postal Code": "Post Code"}) #to match df name

In [ ]:
df2.head()

In [ ]:
# df['Latitude'] = df2['Postal Codes'].map(df2.set_index('Postal Codes')['Latitude'])
            

### Merge after making headers the same

In [ ]:
df = pd.merge(df, df2, on='Post Code', how='outer')
df.head()

# Week 1 Code Battle of the Neighbourhoods

# MOVING ON WITH CODE

> Readign current cases of covid-19 in different boroughs

In [ ]:
df3 = pd.read_csv('/kaggle/input/toronto2/COVID-19Neighbourh.csv')
df3.head()

**43.651070, -79.347015    is coorinates of Toronto**

Change headers to match others

In [ ]:
df3 = df3.rename(columns={"Neighbourhood Name": "Neighbourhood"}) #to match df name

In [ ]:
df3 = df3.rename(columns={"Rate per 100,000 people": "Rate"}) #to match df name

In [ ]:
df3.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#using df3
groupedvalues=df3.groupby('Rate').sum().reset_index()

pal = sns.color_palette("Reds_d", len(groupedvalues))
rank = groupedvalues["Rate"].argsort().argsort() 
tn = 'Neighbourhood'
first_chars = tn[0:3]
g=sns.barplot(x=tn,y='Rate',data=groupedvalues, palette=np.array(pal[::-1])[rank])



plt.show()

In [ ]:
result = df3.sort_values(['Rate'])
result.tail()

In [ ]:
result.head()


Note the huge diference on the InfectionRate per 100,000 people

Add the rate column to df

In [ ]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

* **Being clever**  Attempted to use
df = pd.merge(df, df3, on='Neighbourhood', how='outer')
df = df.dropna() #drop where neighbour hoods disagree 
df
and only got 10 neighbouhood matches so found a fuzzy function as shown below

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

df = fuzzy_merge(df, df3, 'Neighbourhood','Neighbourhood', threshold=80)
#df = pd.merge(df, df3, how='outer')
#df = df.dropna() #drop where neighbour hoods disagree 
df

In [ ]:
print(df.shape)

*The above does not have the data in df3 added if you do we get*

In [ ]:
# definition of the boundaries in the map
district_geo = r'/kaggle/input/toronto/Neighbourhoods.geojson'
  
print (district_geo) #not used using circle size methos instead

In [ ]:
df = pd.merge(df, df3, how='outer')
# df = df.dropna() #drop where neighbourhoods disagree 

In [ ]:
# pip install -U folium
import folium
T_COORDINATES = (43.651070, -79.347015)
mapdata = df
 
# for speed purposes
MAX_RECORDS = 1000
  
# create empty map zoomed in on San Francisco
map = folium.Map(location=T_COORDINATES, zoom_start=12)
 
# add a marker for every record in the filtered data, use a clustered view
# Make an empty map
m = folium.Map(location=T_COORDINATES, zoom_start=12)
 


  
#display(map)

Only 10 neighbourhoods because the niebourhood name in the data taken from the Toronto site are not spelt the same so let us use Torontos dtat set instead

In [ ]:
df4 =  pd.read_csv('/kaggle/input/toronto3/Neighbourhoods.csv')
list(df4)
df4 = df4.drop(['X', 'Y', '_id'], axis=1)
df4.dropna()
df4.head()

In [ ]:
df4 = df4.rename(columns={"AREA_NAME": "Neighbourhood"}) #to match df name
df4 = df4.rename(columns={"LONGITUDE": "Longitude"}) #to match df name
df4 = df4.rename(columns={"LATITUDE": "Latitude"}) #to match df name
df4.head()

In [ ]:
df4['Neighbourhood']=df4['Neighbourhood'].str[:10]



Because of (xx) number in text just compare first 8 characters

In [ ]:
df3['Neighbourhood']=df3['Neighbourhood'].str[:10]
df3

In [ ]:
df4 = pd.merge(df4, df3, on='Neighbourhood', how='outer')
# df4 = df4.apply (pd.to_numeric, errors='coerce')

#df4 = df4.dropna()
df4

In [ ]:

print(df4.shape)

In [ ]:
df4

In [ ]:
# pip install -U folium
import folium
T_COORDINATES = (43.651070, -79.347015)

mapdata = df4

 
# for speed purposes
MAX_RECORDS = 1000
  
# create empty map zoomed in on Toronto
#map = folium.Map(location=T_COORDINATES, zoom_start=12)
 
# add a marker for every record in the filtered data, use a clustered view
# Make an empty map
venues_map = folium.Map(location=T_COORDINATES, zoom_start=12)
 


  
# display(m)

In [ ]:
# I can add marker one by one on the map
import math
for i in range(0,len(mapdata)):    
        location=[mapdata.iloc[i]['Latitude'], mapdata.iloc[i]['Longitude']]
        # print (location[0])
        if (location[0] > 20):
            rate = mapdata.iloc[i]['Rate']
            if (math.isnan(rate)):
                #print(rate)
                rate = 1
            #print (location)
            # print(rate)
            folium.Circle(
              location=[mapdata.iloc[i]['Latitude'], mapdata.iloc[i]['Longitude']],
              popup=mapdata.iloc[i]['Neighbourhood'],
              # radius=mapdata.iloc[i]['Rate']/1,
              radius = rate,
              color='crimson',
              fill=True,
              fill_color='crimson'
       ).add_to(venues_map)
#print(df4.shape)

#display(venues_map)

In [ ]:
import requests # library to handle requests
# import pandas as pd # library for data analsysis
# import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
#from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium # plotting library

#print('Folium installed')
print('Libraries imported.')

In [ ]:
CLIENT_ID = 'BCQ3KNRQKOHYNMOOLYSELTZ2CEJEIDTW1FCABZBDSHDXPMJ3' # My Foursquare ID
CLIENT_SECRET = 'HJ1ZGT1RE2CYHPQGCAPNLNT42YHO3MCJZF3BTWL1ZUG4J3NX' # your Foursquare Secret
VERSION = '20180323'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
address = '88 Boulton Ave, Toronto' 

#geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
#print(location.latitude )
#latitude = location.latitude #43.66278703030303
#longitude = location.longitude # -79.34818218181817

latitude = 43.66278703030303
longitude = -79.34818218181817

print(latitude, longitude)


In [ ]:
search_query = 'Park'
radius = 1000
print(search_query + ' .... OK!')

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
dataframe_filtered.name

In [ ]:
#venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my first address

# add a dark green circle marker to represent my first location
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='darkgreen',
    popup='First Location',
    fill = True,
    fill_color = 'darkgreen',
    fill_opacity = 0.6
).add_to(venues_map)


In [ ]:
# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# ******NEXT start address****





******

In [ ]:
address = '489 Merton St, Toronto' 

#geolocator = Nominatim(user_agent="foursquare_agent")
#location = geolocator.geocode(address)
#print(location.latitude )
#latitude = location.latitude #43.66278703030303
#longitude = location.longitude # -79.34818218181817
#43.69897,-79.38174 Mertonn

latitude = 43.69897
longitude = -79.38174

print(latitude, longitude)


In [ ]:
search_query = 'Park'
radius = 1000
print(search_query + ' .... OK!')

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
dataframe_filtered.name

In [ ]:
#venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my second address
# add a rgreen circle marker to represent my first location
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Second Location',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)


In [ ]:
# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.8
    ).add_to(venues_map)


In [ ]:
venues_map